In [11]:
import os
import re
import json
import pandas as pd

def extract_text_from_sheet(sheet_df):
    text_list = []
    for value in sheet_df.values.flatten():
        if pd.notna(value) and isinstance(value, str):
            # 过滤掉括号及括号中的内容
            value = value.replace('\uff08', '(').replace('\uff09', ')').replace('\uff1a', ':')
            value = value.replace("AK", "").replace("ES", "")
            value = re.sub(r'\(.*?\)', '', value)  # 使用正则表达式去除括号中的内容

            text_list.extend([text.strip() for text in value.split('\n') if text.strip()])
    return text_list

def process_excel_to_json(file_path, output_folder):
    xl = pd.ExcelFile(file_path)
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    specific_output_folder = os.path.join(output_folder, base_name)
    os.makedirs(specific_output_folder, exist_ok=True)
    
    all_text_data = {}
    for sheet_name in xl.sheet_names:
        if "Programming Details" in sheet_name:
            df = xl.parse(sheet_name)
            all_text_data["programming details"] = extract_text_from_sheet(df)
    
    if not all_text_data:
        return None
    
    json_output_path = os.path.join(specific_output_folder, 'input_data.json')
    with open(json_output_path, 'w') as json_file:
        json.dump(all_text_data, json_file, indent=4)
    
    return json_output_path

# 设备类型，针对所有会受到场景操作的设备
DevicesInSceneControl = {
    "Dimmer Type": [
        "KBSKTDIM", "D300IB", "D300IB2", "DH10VIB", 
        "DM300BH", "D0-10IB", "DDAL"
    ],
    "Relay Type": [
        "KBSKTREL", "S2400IB2", "RM1440BH", "KBSKTR", "Z2"
    ],
    "Curtain Type": [
        "C300IBH"
    ],
    "Fan Type": [
        "FC150A2"
    ],
    "RGB Type": [
        "KB8RGBG", "KB36RGBS", "KB9TWG", "KB12RGBD", 
        "KB12RGBG"
    ],
    "PowerPoint Type": {
        "Single-Way": [
            "H1PPWVBX"
        ],
        "Two-Way": [
            "K2PPHB", "H2PPHB", "H2PPWHB"
        ]
    }
}

# 初始化 devices_in_scene_control 字典
devices_in_scene_control = {key: {} if isinstance(value, dict) else [] for key, value in DevicesInSceneControl.items()}
device_name_to_type = {}

def reset_devices_in_scene_control():
    global devices_in_scene_control
    devices_in_scene_control = {key: {} if isinstance(value, dict) else [] for key, value in DevicesInSceneControl.items()}

def reset_device_name_to_type():
    global device_name_to_type
    device_name_to_type = {}

def process_devices(split_data, output_folder):
    devices_content = split_data.get("devices", [])
    devices_data = []
    current_shortname = None
    current_device_name = None  # 用来记录设备的实际名称

    global devices_in_scene_control  # 使用全局变量

    for line in devices_content:
        line = line.strip()

        # 判断是否是新的设备名称
        if line.startswith("NAME:"):
            current_shortname = line.replace("NAME:", "").strip()
            continue
        
        # 跳过 QTY 行
        if line.startswith("QTY:"):
            continue

        # 检查当前设备名称是否包含特定类型的子字符串
        device_type = None
        for dtype, models in DevicesInSceneControl.items():
            if isinstance(models, dict):  # 针对 PowerPoint Type 特殊处理
                for sub_type, sub_models in models.items():
                    for model in sub_models:
                        if model in current_shortname or current_shortname in model:
                            device_type = f"{dtype} ({sub_type})"
                            current_device_name = line.strip()  # 获取设备的实际名称并清理空格
                            devices_in_scene_control[dtype].setdefault(sub_type, []).append(current_device_name)  # 添加到子类型列表
                            break
                    if device_type:
                        break
            else:
                for model in models:
                    if model in current_shortname or current_shortname in model:
                        device_type = dtype
                        current_device_name = line.strip()  # 获取设备的实际名称并清理空格
                        devices_in_scene_control[dtype].append(current_device_name)  # 将设备名称添加到对应的列表中
                        break
            if device_type:
                break

        # 如果有当前设备名称，则添加设备信息
        if current_shortname:
            device_info = {
                "appearanceShortname": current_shortname,
                "deviceName": line
            }
            if device_type:
                device_info["deviceType"] = device_type
            devices_data.append(device_info)

    # 输出 devices.json 文件
    devices_output_path = os.path.join(output_folder, "devices.json")
    with open(devices_output_path, 'w') as file:
        json.dump({"devices": devices_data}, file, indent=4)

    # 输出 devices_in_scene_control.json 文件
    devices_in_scene_control_output_path = os.path.join(output_folder, "devices_in_scene_control.json")
    with open(devices_in_scene_control_output_path, 'w') as file:
        json.dump(devices_in_scene_control, file, indent=4)

    # 构建 device_name_to_type 字典
    global device_name_to_type
    device_name_to_type = {}
    for device_type, device_names in devices_in_scene_control.items():
        if isinstance(device_names, dict):  # 针对 PowerPoint Type 特殊处理
            for sub_type, names in device_names.items():
                for name in names:
                    clean_name = name.strip().upper()  # 清理设备名称并转换为大写
                    device_name_to_type[clean_name] = f"{device_type} ({sub_type})"
                    print(f"Mapping device name '{clean_name}' to type '{device_type} ({sub_type})'")
        else:
            for name in device_names:
                clean_name = name.strip().upper()  # 清理设备名称并转换为大写
                device_name_to_type[clean_name] = device_type
                print(f"Mapping device name '{clean_name}' to type '{device_type}'")

    # 输出 device_name_to_type.json 文件
    device_name_to_type_output_path = os.path.join(output_folder, "device_name_to_type.json")
    with open(device_name_to_type_output_path, 'w') as file:
        json.dump(device_name_to_type, file, indent=4)

def process_groups(split_data, output_folder):
    groups_content = split_data.get("groups", [])
    groups_data = []
    current_group = None

    for line in groups_content:
        line = line.strip()

        # 判断是否是新的组名称
        if line.startswith("NAME:"):
            current_group = line.replace("NAME:", "").strip()
            continue
        
        # 跳过 DEVICE CONTROL: 行
        if line.startswith("DEVICE CONTROL:"):
            continue

        # 如果有当前组名称，则添加设备
        if current_group:
            groups_data.append({
                "groupName": current_group,
                "devices": line
            })

    # 输出 groups.json 文件
    groups_output_path = os.path.join(output_folder, "groups.json")
    with open(groups_output_path, 'w') as file:
        json.dump({"groups": groups_data}, file, indent=4)

scene_output_templates = {
    "Relay Type": lambda name, status: {
        "name": name,
        "status": status,
        "statusConditions": {}
    },
    "Curtain Type": lambda name, status: {
        "name": name,
        "status": status,
        "statusConditions": {
            "position": 100 if status == "OPEN" else 0
        }
    },
    "Dimmer Type": lambda name, status, level=100: {
        "name": name,
        "status": status,
        "statusConditions": {
            "level": level 
        }
    },
    "Fan Type": lambda name, status, relay_status, speed: {
        "name": name,
        "status": status,
        "statusConditions": {
            "relay": relay_status,
            "speed": speed
        }
    },
    "PowerPoint Type": {
        "Two-Way": lambda name, left_power, right_power: {
            "name": name,
            "statusConditions": {
                "leftPowerOnOff": left_power,
                "rightPowerOnOff": right_power
            }
        },
        "Single-Way": lambda name, power: {
            "name": name,
            "statusConditions": {
                "rightPowerOnOff": power
            }
        }
    }
    # 其他类型可以继续添加
}

# 处理函数
def handle_fan_type(parts):
    device_name = parts[0]  # FAN1
    status = parts[1]       # ON or OFF
    relay_status = parts[3] # RELAY ON or OFF
    speed = int(parts[5])   # SPEED 1-3
    return [scene_output_templates["Fan Type"](device_name, status, relay_status, speed)]

def handle_dimmer_type(parts):
    contents = []
    
    # 将输入分成设备条目
    device_entries = " ".join(parts).split(",")
    
    for entry in device_entries:
        entry_parts = entry.strip().split()
        
        if len(entry_parts) < 2:
            continue
        
        device_name = entry_parts[0]  # 获取设备名称
        status = entry_parts[1]       # 获取设备状态
        
        level = 100  # 默认亮度为 100%
        
        if status == "OFF":
            level = 0
        elif status == "ON" and len(entry_parts) > 2:
            # 检查亮度是否在第三个部分
            for part in entry_parts[2:]:
                if part.startswith('+') and '%' in part:
                    try:
                        level = int(part.replace("%", "").replace("+", "").strip())
                    except ValueError:
                        level = 100  # 如果解析失败，默认亮度为 100%
                    break
        
        # 确保 level 被正确包含在 statusConditions 中
        contents.append({
            "name": device_name,
            "status": status,
            "statusConditions": {
                "level": level
            }
        })
    
    return contents

def handle_relay_type(parts):
    status = parts[-1].strip(',').upper() if parts[-1] in ["ON", "OFF"] else parts[-2].strip(',').upper()
    # 拆分设备名称
    device_names = " ".join(parts[:-1] if parts[-1] in ["ON", "OFF"] else parts[:-2]).split(",")
    contents = []
    for device_name in device_names:
        clean_device_name = device_name.strip().strip(',').upper()  # 清理多余字符
        if clean_device_name in device_name_to_type:
            contents.append(scene_output_templates["Relay Type"](clean_device_name, status))
        else:
            raise ValueError(f"无法确定设备类型：{clean_device_name}")
    return contents

def handle_curtain_type(parts):
    status = parts[-1].strip(',').upper() if parts[-1] in ["OPEN", "CLOSE"] else parts[-2].strip(',').upper()
    # 将设备名称部分与状态部分分开处理
    device_names = " ".join(parts[:-1] if parts[-1] in ["OPEN", "CLOSE"] else parts[:-2]).split(",")
    contents = []
    for device_name in device_names:
        clean_device_name = device_name.strip().strip(',').upper()  # 去除多余字符
        if clean_device_name in device_name_to_type:
            contents.append(scene_output_templates["Curtain Type"](clean_device_name, status))
        else:
            raise ValueError(f"无法确定设备类型：{clean_device_name}")
    return contents

def handle_powerpoint_type(parts, device_type):
    contents = []
    
    if "Two-Way" in device_type:
        if len(parts) >= 3:
            device_name = parts[0]
            left_power = parts[1]
            right_power = parts[2]
            contents.append(scene_output_templates["PowerPoint Type"]["Two-Way"](device_name, left_power, right_power))
        else:
            raise ValueError(f"Two-Way PowerPoint '{parts[0]}' input format is incorrect. Expected format: '<device_name> <left_power> <right_power>'. Got: {parts}")
        
    elif "Single-Way" in device_type:
        if len(parts) >= 2:
            device_name = parts[0]
            power = parts[1]
            contents.append(scene_output_templates["PowerPoint Type"]["Single-Way"](device_name, power))
        else:
            raise ValueError(f"Single-Way PowerPoint '{parts[0]}' input format is incorrect. Expected format: '<device_name> <power_state>'. Got: {parts}")
    
    return contents


# 确定设备类型的函数
def determine_device_type(device_name):
    clean_device_name = device_name.strip().strip(',').upper()
    
    if not clean_device_name:  # 如果设备名称为空，跳过或引发异常
        raise ValueError("设备名称不能为空。")

    device_type = device_name_to_type.get(clean_device_name)
    
    if device_type:
        print(f"Device name '{clean_device_name}' identified as '{device_type}'")
        return device_type
    else:
        print(f"Unknown device name: '{clean_device_name}'")
        print("Current device_name_to_type mapping:", device_name_to_type)
        raise ValueError(f"无法确定设备类型：{clean_device_name}")

# 解析场景内容的主函数
def parse_scene_content(scene_name, content_lines):
    contents = []
    
    for line in content_lines:
        parts = line.split()
        if len(parts) < 2:
            continue

        # 判断并处理 Fan Type 设备
        if "RELAY" in line and "SPEED" in line:
            contents.extend(handle_fan_type(parts))
        else:
            # 先确定设备类型，再应用于整行设备
            device_type = determine_device_type(parts[0])
            
            if device_type == "Relay Type":
                contents.extend(handle_relay_type(parts))
            elif device_type == "Curtain Type":
                contents.extend(handle_curtain_type(parts))
            elif device_type == "Dimmer Type":
                contents.extend(handle_dimmer_type(parts))
            elif "PowerPoint Type" in device_type:
                if "Two-Way" in device_type:
                    contents.extend(handle_powerpoint_type(parts, "Two-Way PowerPoint Type"))
                elif "Single-Way" in device_type:
                    contents.extend(handle_powerpoint_type(parts, "Single-Way PowerPoint Type"))
    
    return contents


# 处理整个场景的函数
def process_scenes(split_data, output_folder):
    scenes_content = split_data.get("scenes", [])
    scenes_data = {}
    current_scene = None

    for i, line in enumerate(scenes_content):
        line = line.strip()
        
        if line.startswith("CONTROL CONTENT:"):
            continue
        
        if line.startswith("NAME:"):
            current_scene = line.replace("NAME:", "").strip()
            if current_scene not in scenes_data:
                scenes_data[current_scene] = []
        elif current_scene:
            # 收集当前场景的设备和状态信息
            scenes_data[current_scene].extend(parse_scene_content(current_scene, [line]))

    # 转换场景数据到预期格式
    scenes_output = [{"sceneName": scene_name, "contents": contents} for scene_name, contents in scenes_data.items()]

    # 保存最终数据
    scenes_output_path = os.path.join(output_folder, "scenes.json")
    with open(scenes_output_path, 'w') as file:
        json.dump({"scenes": scenes_output}, file, indent=4)

def process_remote_controls(split_data, output_folder):
    remote_controls_content = split_data.get("remoteControls", [])
    remote_controls_data = []
    current_remote = None
    current_links = []

    for line in remote_controls_content:
        line = line.strip()

        # 跳过 TOTAL 行
        if line.startswith("TOTAL"):
            continue

        # 检查是否是新的远程控制器
        if line.startswith("NAME:"):
            # 如果 current_remote 已经存在，保存它的数据
            if current_remote:
                remote_controls_data.append({
                    "remoteName": current_remote,
                    "links": current_links
                })
            # 解析新的 remoteName
            current_remote = line.replace("NAME:", "").strip()
            current_links = []
        
        # 处理链接部分
        elif line.startswith("LINK:"):
            continue  # 跳过 "LINK:" 行

        # 处理每个链接项
        else:
            parts = line.split(":")
            if len(parts) < 2:
                continue

            link_index = int(parts[0].strip()) - 1  # 0-based index
            link_description = parts[1].strip()

            # 判断 action (TOGGLE, MOMENTARY) 并剥离它
            action = "NORMAL"
            if " - " in link_description:
                link_description, action = link_description.rsplit(" - ", 1)
                action = action.strip().upper()

            # 判断链接类型
            if link_description.startswith("SCENE"):
                link_type = 2  # 场景类型
                link_name = link_description.replace("SCENE", "").strip()
            elif link_description.startswith("GROUP"):
                link_type = 1  # 组类型
                link_name = link_description.replace("GROUP", "").strip()
            elif link_description.startswith("DEVICE"):
                link_type = 0  # 设备类型
                link_name = link_description.replace("DEVICE", "").strip()
            else:
                continue  # 未知的链接类型，跳过

            current_links.append({
                "linkIndex": link_index,
                "linkType": link_type,
                "linkName": link_name,
                "action": action
            })

    # 保存最后一个远程控制器的数据
    if current_remote:
        remote_controls_data.append({
            "remoteName": current_remote,
            "links": current_links
        })

    # 输出 remoteControls.json 文件
    remote_controls_output_path = os.path.join(output_folder, "remoteControls.json")
    with open(remote_controls_output_path, 'w') as file:
        json.dump({"remoteControls": remote_controls_data}, file, indent=4)

def split_json_file(input_file_path, output_folder):
    with open(input_file_path, 'r') as file:
        data = json.load(file)
    content = data.get("programming details", [])
    split_keywords = {
        "devices": "KASTA DEVICE",
        "groups": "KASTA GROUP",
        "scenes": "KASTA SCENE",
        "remoteControls": "REMOTE CONTROL LINK"
    }
    split_data = {
        "devices": [],
        "groups": [],
        "scenes": [],
        "remoteControls": []
    }
    current_key = None
    for line in content:
        if line in split_keywords.values():
            current_key = next(key for key, value in split_keywords.items() if value == line)
            continue
        if current_key:
            split_data[current_key].append(line)
    os.makedirs(output_folder, exist_ok=True)
    
    # 处理设备数据
    process_devices(split_data, output_folder)
    
    # 处理其他数据...
    process_groups(split_data, output_folder)
    process_scenes(split_data, output_folder)
    process_remote_controls(split_data, output_folder)
    
    # 输出 devices_in_scene_control 和 device_name_to_type 文件
    devices_in_scene_control_output_path = os.path.join(output_folder, "devices_in_scene_control.json")
    with open(devices_in_scene_control_output_path, 'w') as file:
        json.dump(devices_in_scene_control, file, indent=4)
    
    device_name_to_type_output_path = os.path.join(output_folder, "device_name_to_type.json")
    with open(device_name_to_type_output_path, 'w') as file:
        json.dump(device_name_to_type, file, indent=4)

def test_process_excel(input_folder, output_folder):
    # 每次测试前先重置 devices_in_scene_control 和 device_name_to_type 字典
    reset_devices_in_scene_control()
    reset_device_name_to_type()
    print("Initialized devices_in_scene_control and device_name_to_type for the test.")

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(input_folder, file_name)
            result = process_excel_to_json(file_path, output_folder)
            if result:
                print(f"Processed {file_name} into {result}")
                split_json_file(result, os.path.dirname(result))
            else:
                print(f"No matching worksheets found in {file_name}")

    # 在每次测试结束后再次重置字典，以防下次测试时数据被继承
    reset_devices_in_scene_control()
    reset_device_name_to_type()
    print("Cleared devices_in_scene_control and device_name_to_type after the test.")

# 设置输入和输出文件夹路径
input_folder = 'test_input'
output_folder = 'test_output'

# 运行测试
test_process_excel(input_folder, output_folder)

Initialized devices_in_scene_control and device_name_to_type for the test.
Processed testing.xlsx into test_output\testing\input_data.json
Mapping device name 'Z1' to type 'Relay Type'
Mapping device name 'Z2' to type 'Relay Type'
Mapping device name 'Z3' to type 'Relay Type'
Mapping device name 'Z4' to type 'Relay Type'
Mapping device name 'Z5' to type 'Relay Type'
Mapping device name 'Z6' to type 'Relay Type'
Mapping device name 'Z7' to type 'Relay Type'
Mapping device name 'Z8' to type 'Relay Type'
Mapping device name 'Z9' to type 'Relay Type'
Mapping device name 'Z10' to type 'Relay Type'
Mapping device name 'Z11' to type 'Relay Type'
Mapping device name 'Z12' to type 'Relay Type'
Mapping device name 'Z13' to type 'Relay Type'
Mapping device name 'FAN' to type 'Relay Type'
Mapping device name 'LIVING_CURTAIN_1' to type 'Curtain Type'
Mapping device name 'LIVING_SHEER_1' to type 'Curtain Type'
Mapping device name 'LIVING_CURTAIN_2' to type 'Curtain Type'
Mapping device name 'LIVING_